In [51]:
# XGBoost CLIP-i embeddingute peal
# Enamik koodi on Joosepi oma, uuendatud on XGBClassifier osa
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score

In [52]:
def labelstring(onehot: np.ndarray) -> str:
    labels = np.array([f'l{i}' for i in range(92)])
    return ' '.join(labels[onehot[0]])

In [53]:
train = np.load('clip_embeddings_train.npz')
X_train = train['embeddings'] 
y_train = train['labels_onehot']

In [54]:
xgboost_model = XGBClassifier(scale_pos_weight=300, max_delta_step=10)
xgboost = MultiOutputClassifier(xgboost_model)
xgboost.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=10, max_depth=None,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))

In [55]:
prediction_on_train = xgboost.predict(X_train)

In [56]:
print(f'f1 score: {f1_score(y_train, prediction_on_train, average="macro"):.3f}')
print(xgboost.score(X_train, y_train))

f1 score: 1.000
1.0


In [57]:
test = np.load('clip_embeddings_test.npz', allow_pickle=True)
X_test = test['embeddings']
test_img_ids = test['img_id']

testdf = pd.read_csv('test.csv')

In [58]:
labelsdf = pd.read_csv('labels.csv')
testlabels = []

for img_id in testdf['image_id']:
    if img_id in test_img_ids:
        prediction = xgboost.predict(test['embeddings'][img_id == test_img_ids].reshape(1,-1))
        predicted_labels = labelstring(prediction.astype(bool))
        if len(predicted_labels) == 0:
            testlabels.append('l1')
        else:
            testlabels.append(predicted_labels)
        print('='*40)
        print(img_id)
        print(labelsdf.loc[labelsdf.label_id.isin(testlabels[-1].split(' ')), 'object'])
        print('='*40)
    else:
        # default label for the missing images in our test set
        print('test image', img_id, 'missing from images')
        testlabels.append('l0')


testdf['labels'] = testlabels
testdf.to_csv('kea_submissions/clip_defaultxgboost_classifier_posw4.csv', index=False)

img102.jpg
1                  trees
5              buildings
24    town hall of tartu
39                 flags
Name: object, dtype: object
img103.jpg
0    people
1     trees
2     grass
7      road
Name: object, dtype: object
img11.jpg
0        people
1         trees
5     buildings
58       worker
63        plane
69    snow plow
Name: object, dtype: object
img113.jpg
10     nature
13    flowers
51     plants
Name: object, dtype: object
img114.jpg
0     people
10    nature
51    plants
Name: object, dtype: object
img121.jpg
0        people
1         trees
2         grass
5     buildings
6         water
19       clouds
22         sand
34        sunny
Name: object, dtype: object
img126.jpg
0                         people
1                          trees
2                          grass
6                          water
38                         barge
40    emajõe-peipsi barge jõmmu
Name: object, dtype: object
img131.jpg
85    stroller
Name: object, dtype: object
img136.jpg
8      perso

In [60]:
testdf['labels'].describe()

count     88
unique    83
top       l0
freq       4
Name: labels, dtype: object